# Day 04 of 14 Days Databricks Challenge


In [0]:
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)


In [0]:
events.write \
    .format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/ecommerce/ecommerce_data/events_delta")


In [0]:
events.write.format("delta").saveAsTable("events_table")


In [0]:
%sql
SELECT * FROM events_table;


In [0]:
%sql
CREATE TABLE events_delta
USING DELTA
AS SELECT * FROM events_table


In [0]:
try:
    wrong_schema = spark.createDataFrame(
        [("a","b","c")],
        ["x","y","z"]
    )
    wrong_schema.write \
        .format("delta") \
        .mode("append") \
        .save("/Volumes/workspace/ecommerce/ecommerce_data/events_delta")
except Exception as e:
    print(f"Schema enforcement: {e}")


In [0]:
from delta.tables import DeltaTable

events = spark.read.format("delta").load(
    "/Volumes/workspace/ecommerce/ecommerce_data/events_delta"
)
delta_table = DeltaTable.forPath(spark, "/Volumes/workspace/ecommerce/ecommerce_data/events_delta")

delta_table.alias("target").merge(
    events.alias("source"),
    "target.user_session = source.user_session AND target.event_time = source.event_time"
).whenNotMatchedInsertAll().execute()

In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(
    spark,
    "/Volumes/workspace/ecommerce/ecommerce_data/events_delta"
)

delta_table.history().show(truncate=False)